In [1]:
import os
import nltk

from gensim import corpora, models
from ipywidgets import IntProgress
from IPython.display import display

Read the files.

In [2]:
def srt_to_strs(f):
    '''Convert a .srt file to a string'''
    text_lines = []
    while True:
        _seq_no = f.readline().strip()
        if _seq_no == '': break
        _time_str = f.readline()
        
        while True:
            text_line = f.readline().strip()
            if text_line == '': break
            text_line = text_line.replace('&#39;', "'")
            text_line = text_line.replace('&gt;', '')
            text_line = text_line.replace('[inaudible]', '')
            text_line = text_line.replace('gonna', 'going to')
            text_lines.append(text_line)
            
    return ' '.join(text_lines)

DATA_DIR = '../../data/'
CAPTIONS_DIR = DATA_DIR + 'compilers_captions/'
SRT_FILE_NAMES = [CAPTIONS_DIR + file_name for file_name in os.listdir(CAPTIONS_DIR) if file_name.endswith('.srt')]

docs = []
tokenized_docs = []

for name in SRT_FILE_NAMES:
    with open(name) as f:
        doc = unicode(srt_to_strs(f), 'utf-8').lower()
        docs.append(doc)
        doc = doc.replace("'", '')
        tokenized_docs.append([nltk.word_tokenize(sent) for sent in nltk.sent_tokenize(doc)])
        


# for doc in documents:
#     sents = [nltk.word_tokenize(sent) for sent in nltk.sent_tokenize(doc)]
#     tags.extend(tagger.tag_sents(sents))
#     prog.value += 1
        
        
# # Remove stopwords
# stopwords = set()
# with open('/Users/andrewlamb/Google_Drive/Stanford/CS199/RAKE-tutorial/SmartStoplistAdditional.txt') as f:
#     for line in f:
#         stopwords.add(line.strip())
        
#     stopwords = stopwords.union({',', '.', '?', '!'})
        
# stopped_tokens = [[tok for tok in toks if tok not in stopwords] for toks in tokens]

# stemmer = nltk.stem.PorterStemmer()
# stemmed_docs = [[stemmer.stem(token) for token in doc] for doc in stopped_tokens]

# dictionary = corpora.Dictionary(stemmed_docs)
# corpus = [dictionary.doc2bow(text) for text in stemmed_docs]

# ldamodel = models.ldamodel.LdaModel(corpus, num_topics=20, id2word=dictionary, passes=100)

In [3]:
# POS Tag
pos_docs = []
prog = IntProgress(min=0, max=len(docs))
display(prog)

os.environ['CLASSPATH'] = '/Users/andrewlamb/Downloads/stanford-postagger-2015-12-09'
os.environ['STANFORD_MODELS'] = '/Users/andrewlamb/Downloads/stanford-postagger-2015-12-09'
tagger = nltk.tag.StanfordPOSTagger('models/english-bidirectional-distsim.tagger')

for doc in tokenized_docs:
    pos_docs.append(tagger.tag_sents(doc))
    prog.value += 1

In [5]:
def extract_tags(tree):
    results = []
    
    if type(tree) == nltk.tree.Tree:
        if tree.label() == 'NP':
            phrase = ' '.join([word for word, _pos in tree])
            phrase = phrase.lower()
            results.append(phrase)

        for child in tree:
            results.extend(extract_tags(child))
    
    return results

grammar = '''
NP: {(<JJ>|<JJS>|<JJR>)*(<NN>|<NNS>|<NNP>|<NNPS>)+} 
'''

cp = nltk.RegexpParser(grammar)
candidates = set()

chunked_docs = []

for doc in pos_docs:
    chunked_doc = []
    for sent in doc:
        result = cp.parse(sent)
        chunked_doc.extend(extract_tags(result))
    chunked_docs.append(chunked_doc)
    
# Remove stopwords
stopwords = set()
with open('/Users/andrewlamb/Google_Drive/Stanford/CS199/RAKE-tutorial/SmartStoplistAdditional.txt') as f:
    for line in f:
        stopwords.add(line.strip())
stopped_docs = []
for doc in chunked_docs:
    stopped_docs.append([token for token in doc if len(token) > 2 and token not in stopwords])

stemmer = nltk.stem.PorterStemmer()
stemmed_docs = []
for doc in stopped_docs:
    stemmed_doc = []
    for chunk in doc:
        stemmed_doc.append(' '.join([stemmer.stem(word) for word in nltk.word_tokenize(chunk)]))

    stemmed_docs.append(stemmed_doc)

    
dictionary = corpora.Dictionary(stemmed_docs)
corpus = [dictionary.doc2bow(text) for text in stemmed_docs]

ldamodel = models.ldamodel.LdaModel(corpus, num_topics=20, id2word=dictionary, passes=10)

In [12]:
dictionary[1]

u'pre-process step'

In [43]:
from pprint import pprint
pprint(ldamodel.print_topics(num_topics=20, num_words=20))

[(0,
  u'0.078*state + 0.044*input + 0.033*machin + 0.023*start state + 0.022*final state + 0.022*set + 0.021*transit + 0.019*languag + 0.019*nfa + 0.014*dfa + 0.012*string + 0.012*regular express + 0.009*automaton + 0.008*java + 0.008*end + 0.007*epsilon closur + 0.007*case + 0.006*time + 0.006*fact + 0.006*finit automaton'),
 (1,
  u'0.073*object + 0.045*pointer + 0.024*program + 0.016*point + 0.012*old space + 0.012*loop + 0.012*new space + 0.011*memori + 0.010*reachabl object + 0.009*assign + 0.009*statement + 0.009*space + 0.007*string + 0.007*time + 0.007*thing + 0.007*garbag collect + 0.006*copi + 0.006*keyword + 0.006*forward pointer + 0.006*futur'),
 (2,
  u'0.047*code + 0.026*string + 0.018*list + 0.016*program + 0.014*express + 0.011*time + 0.011*instruct + 0.009*case + 0.008*assign + 0.008*thing + 0.008*optim + 0.007*result + 0.007*languag + 0.006*compil + 0.006*item + 0.006*constant + 0.006*accumul + 0.005*regist + 0.005*number + 0.005*predic'),
 (3,
  u'0.027*optim + 0.02

In [29]:
dictionary = corpora.Dictionary(stemmed_docs)
corpus = [dictionary.doc2bow(text) for text in stemmed_docs]

157